In [ ]:
import sys
import dask
import numpy as np
import dask.array as da
import plotly.graph_objects as go
from scipy.stats import pearsonr
from dask.diagnostics import ProgressBar

sys.path.append('/home/austinbaggetta/csstorage3/CircleTrack/CircleTrackAnalysis')
import plotting_functions as pf

def generate_poisson_spikes(rate, T):
    """
    Generate spike times for a neuron with a constant firing rate using a Poisson process.
    
    Parameters:
    rate (float): Firing rate (spikes per second).
    T (float): Total duration of the simulation (seconds).
    
    Returns:
    spike_times (list): List of spike times.
    """
    spike_times = []
    t = 0
    
    while t < T:
        # Generate the next spike interval
        interval = -np.log(np.random.rand()) / rate
        t += interval
        
        if t < T:
            spike_times.append(t)
    
    return np.asarray(spike_times)

## Set seed for reproducibility
np.random.seed(24601)
nshuffles = 5000
num_spatial_bins = 20
session_length = 60 ## in seconds

In [37]:
## Generate uniformly spiking neuron and occupancy map
neuron = generate_poisson_spikes(rate=5, T=session_length)
time_in_bin = np.random.normal(loc=200, scale=50, size=num_spatial_bins) 
occupancy = time_in_bin / np.sum(time_in_bin)


In [46]:
time_in_bin

array([178.92514516, 150.23149509, 197.8875793 , 216.30926119,
       152.99204868, 168.27794917, 214.13020202, 182.60541051,
       248.28928414, 323.12635992, 194.79316808, 250.40484483,
       236.09501164, 176.99415586, 156.75041751, 122.85917269,
       213.00567496, 197.21640236, 137.12152026, 275.63452885])

In [ ]:
## Create two arrays to correlate
ar = np.random.normal(size=ar_size)
stable_ar = np.random.normal(size=ar_size[1])

In [ ]:
dar = da.from_array(ar, chunks=(30, -1))
dar

In [ ]:
## Faster shuffling with dask?
for shuffle in np.arange(0, nshuffles):
    rolled = shuffle_data(ar)
    cor = return_pearsonr(rolled, stable_ar)
    shuffled_ar = combine_shuffled_r_vals(cor, nshuffles, shuffle)

In [ ]:
shuffled_ar.visualize()